In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.linalg import orthogonal_procrustes
from sklearn.metrics import mean_squared_error
from helpers_pmf import *
from helpers_similarity import *
from helpers_optimization import *

In [56]:
import pandas as pd

# Generate the tensor Y
mu = 0
sigma_u = 1
sigma_v = 1
#sigma = 0.1
d_dim = 2
n_users = 3
n_movies = 4

In [57]:
def generate_U_V_X(mu, sigma_u, sigma_v, d_dim, n_users, n_movies):
    U = np.random.normal(mu, sigma_u, size = (d_dim,n_users))
    V = np.random.normal(mu, sigma_v, size = (d_dim,n_movies))
    #noise = np.random.normal(mu, sigma, size = (n_users,n_movies)) 
    X = np.matmul(U.T,V) #+noise
    return U, V, X 

In [94]:
def generate_Y_BT_Luce(X):
    # Compute the pairwise differences
    diff = np.subtract.outer(X, X) # reduce dimension
    # Compute the exponentiated differences
    #print(diff)
    diff_3D= np.array([diff[i, :, i, :] for i in range(n_users)])
    P = np.exp(diff_3D) / (1 + np.exp(diff_3D))
    return P, diff_3D


In [95]:
U, V, X = generate_U_V_X(mu, sigma_u, sigma_v, d_dim, n_users, n_movies)

In [111]:
print('X : ',X)
P, diff = generate_Y_BT_Luce(X)
print('P : ',P)
#diff=diff[0]
#Y=Y[0]
i=2
j=1
k=0
print(P[i,j,k])
print(diff[i,j,k])
print(X[i,j]-X[i,k])

X :  [[ 3.59634129  1.06974362  2.44227835 -0.76519373]
 [ 1.53699722 -0.20568413  0.67781367 -0.51294693]
 [ 3.34658903  2.79979582  3.26882788 -0.20597638]]
P :  [[[0.5        0.92598551 0.76025225 0.98740195]
  [0.07401449 0.5        0.20221063 0.86234886]
  [0.23974775 0.79778937 0.5        0.9611145 ]
  [0.01259805 0.13765114 0.0388855  0.5       ]]

 [[0.5        0.85102733 0.70249005 0.88594198]
  [0.14897267 0.5        0.29245347 0.576217  ]
  [0.29750995 0.70754653 0.5        0.76687707]
  [0.11405802 0.423783   0.23312293 0.5       ]]

 [[0.5        0.63339127 0.5194305  0.97214697]
  [0.36660873 0.5        0.38484537 0.95283422]
  [0.4805695  0.61515463 0.5        0.96996231]
  [0.02785303 0.04716578 0.03003769 0.5       ]]]
0.366608729894068
-0.546793203163352
-0.546793203163352


In [113]:
Y = np.random.binomial(n=1, p=P, size=P.shape) 
Y = np.where(Y == 0, -1, Y)
Y

(3, 4, 4)